In [1]:
import sqlite3

conn=sqlite3.connect("nominations.db")

In [2]:
schema=conn.execute("pragma table_info(nominations);").fetchall()

In [3]:
first_ten=conn.execute("select * from nominations limit 10;").fetchall()

In [5]:
for s in schema:
    print(s)
    
print()
print()
for row in first_ten:
    print(row)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)


(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supportin

In [7]:
query="""create table ceremonies(
id integer primary key,
Year integer,
Host text);
"""
conn.execute(query)

In [10]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]
insert_query="insert into ceremonies(Year,Host) values(?,?);"

conn.executemany(insert_query,years_hosts)

NameError: name 'quer' is not defined

In [14]:
schema_ceremonies=conn.execute("pragma table_info(ceremonies);").fetchall()
first_ten_ceremonies=conn.execute("select * from ceremonies limit 10;").fetchall()

In [15]:
for s in schema_ceremonies:
    print(s)
    
print()
print()
for row in first_ten_ceremonies:
    print(row)

(0, 'id', 'integer', 0, None, 1)
(1, 'Year', 'integer', 0, None, 0)
(2, 'Host', 'text', 0, None, 0)


(1, 2010, 'Steve Martin')
(2, 2009, 'Hugh Jackman')
(3, 2008, 'Jon Stewart')
(4, 2007, 'Ellen DeGeneres')
(5, 2006, 'Jon Stewart')
(6, 2005, 'Chris Rock')
(7, 2004, 'Billy Crystal')
(8, 2003, 'Steve Martin')
(9, 2002, 'Whoopi Goldberg')
(10, 2001, 'Steve Martin')


In [16]:
conn.execute("PRAGMA foreign_keys = ON;")

In [18]:
create_query="""create table nominations_two
(id integer primary key,
category text,
nominee text,
movie text,
character text,
won integer,
ceremony_id integer,
foreign key(ceremony_id) references ceremonies(id));"""

conn.execute(create_query)

In [19]:
joined_query="""
SELECT nominations.category, nominations.nominee, nominations.movie, nominations.character, nominations.won, ceremonies.id
FROM nominations
INNER JOIN ceremonies ON
nominations.year == ceremonies.year
;"""

joined_nominations=conn.execute(joined_query).fetchall()

In [22]:
insert_nominations_two="""
insert into nominations_two(category,nominee,movie,character,won,ceremony_id) values(?,?,?,?,?,?)"""

conn.executemany(insert_nominations_two,joined_nominations)

In [23]:
conn.execute("select * from nominations_two limit 5;").fetchall()

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1),
 (2,
  'Actor -- Leading Role',
  'Jeff Bridges',
  'True Grit',
  'Rooster Cogburn',
  0,
  1),
 (3,
  'Actor -- Leading Role',
  'Jesse Eisenberg',
  'The Social Network',
  'Mark Zuckerberg',
  0,
  1),
 (4,
  'Actor -- Leading Role',
  'Colin Firth',
  "The King's Speech",
  'King George VI',
  1,
  1),
 (5,
  'Actor -- Leading Role',
  'James Franco',
  '127 Hours',
  'Aron Ralston',
  0,
  1)]

In [24]:
drop_query="DROP TABLE nominations;"
conn.execute(drop_query)

In [25]:
alter_query="alter table nominations_two rename to nominations;"
conn.execute(alter_query)

In [26]:
table_list=[]

movies_query="""
create table movies (id integer primary key,
movie text);"""
table_list.append(movies_query)

actors_query="""
create table actors (id integer primary key,
actor text);"""
table_list.append(actors_query)

movies_actors="""
create table movies_actors(id integer primary key,
movie_id integer,
actor_id integer,
foreign key(movie_id) references movies(id),
foreign key(actor_id) references actors(id))"""

table_list.append(movies_actors)

In [27]:
for l in table_list:
    conn.execute(l)

Here are some ideas:

What other datasets can we add to the database?
Based on what you know, brainstorm how you would populate the join table and the linked tables using data from nominations.

In [28]:
insert_movies=conn.execute("select movie from nominations;").fetchall()
insert_actor=conn.execute("select character from nominations;").fetchall()

conn.executemany("insert into movies(movie) values(?)",insert_movies)
conn.executemany("insert into actors(actor) values(?)",insert_actor)

In [32]:
joined_query="""
select movies.id,actors.id from nominations
inner join movies
on movies.movie == nominations.movie
inner join actors
on actors.actor == nominations.character"""
joined_result=conn.execute(joined_query).fetchall()
conn.executemany("insert into movies_actors(movie_id,actor_id) values(?,?)",joined_result)